# Podstawy Sztucznej Inteligencji 2020/2021


Prosze uzupelnic kod tam gdzie znajduje napis `YOUR CODE HERE` lub 'YOUR ANSWER HERE'.

Warto zresetowac 'kernel' i sprawdzic czy caly notatnik uruchamiany od poczatku nie daje bledow.

---

# Naiwny Bayes

Notatnik ten zawiera implementacją naiwnego klasyfikatowa Bayesa wykorzystując bibliotekę `numpy`. Działanie klasyfikatora jest porównane implementacja ``GaussianNB`` w bibliotece `sklearn`.

Z twierdzenia Bayesa mamy:

$$P(J=j|X=x)=\frac{P(X=x|J=j)P(J=j)}{\sum_{j\in\{1,\ldots L\}}P(X=x|J=j)P(J=j)}$$

$$ p_j(x)=\frac{p_jf_j(x)}{f(x)} $$

Wykorzystamy te formuły w budowie klasyfikatora [Bayesa](http://books.icse.us.edu.pl/runestone/static/ai/KlasyfikacjaWOparciuPodobienstwoDoWzorca/OptymalnyKlasyfikatorStatystyczny.html).

In [4]:
import numpy as np 
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn import datasets 
# dataset import 
iris = datasets.load_iris()

gnb = GaussianNB()

X = iris.data

y = iris.target
y = np.array(y)

train, test, train_targets, test_targets = \
   train_test_split(X, y, test_size=0.31)

# train 
clf = gnb.fit(train, train_targets)

# Test
Z = clf.predict(test)

In [5]:
train.shape,test.shape

((103, 4), (47, 4))

### Wyznacz unikalne klasy na zbiorze trenującym.

In [6]:
Nlabels = np.unique(train_targets)

In [ ]:
np.testing.assert_array_equal(Nlabels,np.array([0, 1, 2]))

### Jaki jest procent poprawych odpowiedzi?

Implementacja ``GaussianNB`` daje tę odpowiedź w następujący sposób. Spróbuj otrzymać tą liczbę korzystając wyłącznie z funkcjonalności ``numpy`` i/lub języka Python.

In [7]:
clf.score(test,test_targets)

0.9787234042553191

In [8]:
a = np.sum(test_targets == Z)
b = np.size(Z)
correct = a/b

In [9]:
assert correct==clf.score(test,test_targets)

### Które odpowiedzi są złe:

In [10]:
bad_idx = np.where(Z!=test_targets)
bad_idx

(array([27]),)

### Parametry klasyfikatora

In [11]:
clf.theta_

array([[5.01      , 3.42      , 1.48333333, 0.24      ],
       [5.9025    , 2.77      , 4.2425    , 1.3275    ],
       [6.41212121, 2.93939394, 5.4030303 , 2.0030303 ]])

In [12]:
clf.sigma_

array([[0.12756667, 0.13493334, 0.02405556, 0.01306667],
       [0.23424375, 0.0871    , 0.22694375, 0.03749375],
       [0.34045914, 0.08541782, 0.24150597, 0.08271809]])

In [13]:
ith = 2
np.mean(train[train_targets == ith],axis=0)

array([6.41212121, 2.93939394, 5.4030303 , 2.0030303 ])

In [14]:
np.var(train[train_targets == ith],axis=0)

array([0.34045914, 0.08541781, 0.24150597, 0.08271809])

## Implementacja naiwnego klasyfikatora Bayesa

**1\. Oblicz częstość występowania poszczególnych klasy $j$ w zbiorze treningowym $p_{j}$**

Jest to prawdopodobieństwo *a priori*.

In [15]:
p = np.unique(train_targets, return_counts = True)[1]/train_targets.shape[0]
#druga metoda:
#p = [np.sum(train_targets == ith)/train_targets.shape[0] for ith in range(3)]

In [16]:
np.testing.assert_allclose(p,clf.class_prior_)

**2\. Oblicz wartość średnią dla każdej cechy z każdej klasy.**

Niech $\mu_{ij}$ oznacza  wartość średnią dla $j$-tej zmiennej w $i$-tej klasie, wtedy:
   
$$ \mu_{ij} =  \langle x_j \rangle_{ \forall x_j \in \;\mathrm{label} \;
i } $$

In [17]:
mu = np.stack([np.mean(train[train_targets == ith], axis=0) for ith in range(3)])

In [18]:
np.testing.assert_allclose(mu,clf.theta_)

In [19]:
mu.shape

(3, 4)

**3\. Oblicz wariancję $j$-tej zmiennej w $i$-tej klasie.**

   $$ \sigma_{ij}^2 = \mathrm{Var} [x_j] _ {\;\;{ \forall x_j \in \mathrm{class}\; i}} $$

In [38]:
sigma2 = np.stack([np.var(train[train_targets == ith], axis=0) for ith in range(3)])

In [39]:
np.testing.assert_almost_equal(sigma2,clf.sigma_)

**4\. Oblicz prawdopodobieństwo *a posteriori* klasy $i$ dla danego wektora zmiennych  $\mathbf{x}$.**

Niech $k$ oznacza liczbę cech (zmiennych). W naszym przypadku mamy $k=4$.

Dla $i$-tej klasy mamy:


$$ P_i(\mathbf{x}) \simeq p_i f_i = p_i \frac{1}{\sqrt{(2\pi)^k\Pi_{j=1}^k\sigma_{ij}^2} } e^{ -\displaystyle\sum_{j=1}^{k}\frac{(x_j-\mu_{ij})^2}{2\sigma_{ij}^2} }
 $$
 
By otrzymać prawdopodobieństwa należy unormować $P_i$ dla każdego przypadku tak by suma $\sum_i P_i(\mathbf{x})=1$

In [47]:
x  = test
P = None
k = test.shape[1] 

print ('number of features: k=',k)
#OK!!!
P = np.stack([p[i]*(1/(np.sqrt((2*np.pi)**k*np.prod(sigma2[i]))))*np.exp(-1/2.0*np.sum((x-mu[i])*(x-mu[i])/sigma2[i], axis=1)) for i in range(3)]).T
P = P/np.sum(P, axis = 1)[:, np.newaxis]

#???
#P = p/np.sqrt((2*np.pi)**k * np.prod(sigma2[i], axis=1))*np.exp(-0.5*np.sum(x[:, np.newaxis, :] - mu[np.newaxis, :, :])**2/sigma2[np.newaxis, :, :], axis=2)
#P = P/np.sum(P, axis=1)[:,newaxis]


number of features: k= 4


In [48]:
sigma2.shape,x.shape,P.shape,np.unique(test_targets)

((3, 4), (47, 4), (47, 3), array([0, 1, 2]))

In [49]:
np.testing.assert_almost_equal(P,clf.predict_proba(x))

**5\. Wyznacz klasę dla której prawdopodobieństwo *a posteriori* jest największe** 

 $i$ : $\quad\textrm{ gdy } P_i(x)=\max_{1\leq j\leq L} P_j(x)$


In [56]:
prediction = np.argmax(P, axis=1)

In [57]:
np.testing.assert_equal( prediction, clf.predict(test))

In [58]:
print(clf.predict(test) )
print(prediction)

[1 0 2 2 0 0 0 0 2 0 0 2 0 2 2 0 2 2 0 2 0 0 2 2 1 0 1 2 2 0 1 1 1 2 1 0 2
 0 0 0 1 0 1 0 2 2 2]
[1 0 2 2 0 0 0 0 2 0 0 2 0 2 2 0 2 2 0 2 0 0 2 2 1 0 1 2 2 0 1 1 1 2 1 0 2
 0 0 0 1 0 1 0 2 2 2]


In [59]:
clf.score(test,test_targets)

0.9787234042553191

In [60]:
clf.predict(test) == prediction

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])